In [1]:
import pandas as pd

In [4]:
metadataindir="/Shares/down/public/HTP/RNAseq/inputdata/"
participanttable="include-participant-table-2023-08-22.tsv"
count_dir = "/Shares/down/public/HTP/RNAseq/outputdata/T21vsD21_noDNAdosagecorrection/"
count_file = count_dir+"normcounts.csv"
outdir = "/Shares/down/public/INLCUDE_2024/kallisto_20241030/selfannoated/"
geneindir= outdir
genefile = geneindir+"genes.csv"

In [5]:
metadatadf = pd.read_csv(metadataindir+participanttable, sep="\t")


In [6]:
def createbinary(metadatadf, importantcol, spliton, findcode, appendname):
    allphenotypes = metadatadf[importantcol].to_list()
    allphenotypes = [v.split(")") for v in allphenotypes]
    allphenotypes = [item.strip(", ") for sublist in allphenotypes for item in sublist]
    allphenotypes = sorted(list(set([v for v in allphenotypes if v.find(findcode)>-1])))
    df = pd.DataFrame(allphenotypes)
    df.columns = ["ori_name"]
    df[["phenotype", appendname]] = df["ori_name"].str.split(spliton, expand=True)
    df["colname"] = appendname+"_"+df["phenotype"].str.replace(" ", "_")
    new_columns = {}

    for _, row in df.iterrows():
        # Get the HP value and colname
        hp_value = row[appendname]
        colname = row['colname']
        
        # Apply the logic to check if the HP value is in the Phenotype (HPO) column
        if appendname=="HP":
            new_columns[colname] = metadatadf[importantcol].apply(
                lambda x: 1 if f"(HP:{hp_value})" in x else 0
            )
        if appendname=="MONDO":
            new_columns[colname] = metadatadf[importantcol].apply(
                lambda x: 1 if f"(MONDO:{hp_value})" in x else 0
            )

    # Concatenate the nenewmetadatadfw columns to metadatadf
    newmetadatadf = pd.concat([metadatadf, pd.DataFrame(new_columns)], axis=1)
    bianary_table = newmetadatadf[["Participant ID"]+list(new_columns.keys())]
    bianary_table.index = bianary_table["Participant ID"]
    bianary_table = bianary_table[list(new_columns.keys())]
    counteach = bianary_table.sum().to_frame()
    counteach.columns = ["counts"]
    counteach["colname"]= counteach.index
    df = df.merge(counteach, on="colname")
    return df, bianary_table

In [7]:
importantcol = "Diagnosis (Mondo)"
spliton = " \(MONDO:"
findcode = " (MONDO:"
appendname = "MONDO"
mondodf, mondobianary_table = createbinary(metadatadf, importantcol, spliton, findcode, appendname)

In [8]:
importantcol = "Phenotype (HPO)"
spliton = " \(HP:"
findcode = " (HP:"
appendname = "HP"
HPdf, HPbianary_table = createbinary(metadatadf, importantcol, spliton, findcode, appendname)

In [9]:
HPbianary_table["Participant"] = HPbianary_table.index
mondobianary_table["Participant"] = mondobianary_table.index

In [10]:
HPbianary_table = HPbianary_table.reset_index(drop=True)
mondobianary_table = mondobianary_table.reset_index(drop=True)
HPbianary_table = HPbianary_table[[colname for colname in HPbianary_table.columns if colname!="Participant ID"]]
mondobianary_table = mondobianary_table[[colname for colname in mondobianary_table.columns if colname!="Participant ID"]]


In [11]:
#HPdf.to_csv(outdir+"full_HP_counts_colnames.csv")
#HPbianary_table.to_csv(outdir+"full_HP_binary_attribute.csv")
#mondodf.to_csv(outdir+"full_MONDO_counts_colnames.csv")
#mondobianary_table.to_csv(outdir+"full_MONDO_binary_attribute.csv")

In [21]:
HPbianary_table.columns.to_list()

['HP_Abnormal_duodenum_morphology',
 'HP_Abnormal_heart_morphology',
 'HP_Abnormal_subclavian_artery_morphology',
 'HP_Abnormality_of_pulmonary_circulation',
 'HP_Abnormality_of_the_Eustachian_tube',
 'HP_Abnormality_of_the_eye',
 'HP_Abnormality_of_the_gastrointestinal_tract',
 'HP_Abnormality_of_the_genitourinary_system',
 'HP_Abnormality_of_the_nervous_system',
 'HP_Abnormality_of_the_respiratory_system',
 'HP_Abnormality_of_the_skeletal_system',
 'HP_Abnormality_of_the_skin',
 'HP_Abnormality_of_the_thyroid_gland',
 'HP_Acne_inversa',
 'HP_Acute_lymphoblastic_leukemia',
 'HP_Acute_myeloid_leukemia',
 'HP_Aganglionic_megacolon',
 'HP_Alzheimer_disease',
 'HP_Amblyopia',
 'HP_Anomalous_pulmonary_venous_return',
 'HP_Anti-thyroid_peroxidase_antibody_positivity',
 'HP_Anxiety',
 'HP_Aortic_regurgitation',
 'HP_Arrhythmia',
 'HP_Aspiration',
 'HP_Asthma',
 'HP_Astigmatism',
 'HP_Atrial_fibrillation',
 'HP_Atrial_septal_defect',
 'HP_Atrioventricular_canal_defect',
 'HP_Attention_deficit

In [22]:
mini_HP = HPbianary_table[["HP_Obstructive_sleep_apnea", "HP_Obesity","HP_Depression", "Participant"]]
mini_HP.to_csv(outdir+"mini_HP_binary_attribute.csv")

In [23]:
countdf = pd.read_csv(count_file, index_col=0)

value_df = countdf.T



In [24]:
value_df["Participant"] = value_df.index
value_df = value_df.reset_index(drop=True)
value_df = value_df[[colname for colname in value_df.columns if colname!="Participant ID"]]


In [27]:
#value_df.to_csv(outdir+"RNAvalues_normcounts.csv")
mini_value_df = value_df[["Participant", "ENSG00000124813", "ENSG00000159216", "ENSG00000001167"]]
mini_value_df.to_csv(outdir+"miniRNAvalues_normcounts.csv")

In [80]:
participantswithRNA= list(value_df["Participant"])
participantswithconditions = list(mondobianary_table["Participant"])


participantswithRNA_notconditionstable = [participant for participant in participantswithRNA 
                                    if participant not in participantswithconditions]

participantswithRNA_conditions = [participant for participant in participantswithRNA 
                                    if participant in participantswithconditions]
 

In [81]:
len(participantswithRNA_notconditionstable)

0

In [82]:
len(participantswithRNA_conditions)

400

In [83]:
len(participantswithRNA)

400

In [84]:
len(participantswithconditions)

700

In [115]:
participantswithRNA = pd.DataFrame(participantswithRNA)
participantswithRNA.to_csv(outdir+"include_participants_with_RNA.csv", header=False, index=False)


In [85]:
DS_conditions_cols = ["MONDO_Down_syndrome", "MONDO_mosaic_translocation_Down_syndrome", "MONDO_translocation_Down_syndrome", "MONDO_complete_trisomy_21", "MONDO_mosaic_trisomy_21"]



In [87]:
whichT21 = mondobianary_table[DS_conditions_cols].copy()
whichT21.index = mondobianary_table["Participant"]

In [88]:
whichT21.loc[:, "Disomic"] = whichT21[DS_conditions_cols].sum(axis=1) == 0


In [89]:
whichT21.to_csv(outdir+"whichT21.csv")

In [108]:
completeT21 =  whichT21[whichT21["MONDO_complete_trisomy_21"]==1]

In [109]:
completeT21

,MONDO_Down_syndrome,MONDO_mosaic_translocation_Down_syndrome,MONDO_translocation_Down_syndrome,MONDO_complete_trisomy_21,MONDO_mosaic_trisomy_21,Disomic
Participant,,,,,,
pt-01qcpze4,0,0,0,1,0,False
pt-049kaxj5,0,0,0,1,0,False
pt-081x47b5,0,0,0,1,0,False
pt-08qxc61x,0,0,0,1,0,False
pt-0e3fehnr,0,0,0,1,0,False
...,...,...,...,...,...,...
pt-zdchddxn,0,0,0,1,0,False
pt-zmqyrma5,0,0,0,1,0,False
pt-zpk50nrb,0,0,0,1,0,False


In [116]:
participantswithRNAandcompleteT21 = participantswithRNA[participantswithRNA[0].isin(completeT21.index)]
participantswithRNAandcompleteT21.to_csv(outdir+"include_participants_with_RNA_and_completeT21.csv", header=False, index=False)


In [46]:
genesdf = pd.read_csv(genefile)
genesdf

,Unnamed: 0,seqnames,start,end,width,strand,source,type,gene_id,gene_version,gene_name,gene_source,gene_biotype
0,1,1,11869,14409,2541,+,havana,gene,ENSG00000223972,5,DDX11L1,havana,transcribed_unprocessed_pseudogene
1,2,1,14404,29570,15167,-,havana,gene,ENSG00000227232,5,WASH7P,havana,unprocessed_pseudogene
2,3,1,17369,17436,68,-,mirbase,gene,ENSG00000278267,1,MIR6859-1,mirbase,miRNA
3,4,1,29554,31109,1556,+,havana,gene,ENSG00000243485,5,MIR1302-2HG,havana,lincRNA
4,5,1,30366,30503,138,+,mirbase,gene,ENSG00000284332,1,MIR1302-2,mirbase,miRNA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58730,58731,KI270711.1,4612,29626,25015,-,ensembl,gene,ENSG00000271254,6,AC240274.1,ensembl,protein_coding
58731,58732,KI270713.1,21861,22024,164,-,ensembl,gene,ENSG00000275405,1,RF00003,ensembl,snRNA
58732,58733,KI270713.1,30437,30580,144,-,ensembl,gene,ENSG00000275987,1,RF00003,ensembl,snRNA
58733,58734,KI270713.1,31698,32528,831,-,ensembl,gene,ENSG00000277475,1,AC213203.2,ensembl,protein_coding


In [47]:
genesdfchr21 = genesdf[genesdf["seqnames"]=="21"]
genesdfchr21.shape

(833, 13)

In [117]:
genenamechr21 = genesdfchr21[["gene_id"]].copy()
genenamechr21.to_csv(outdir+"include_rank_file_chr21only.csv", header=False, index=False)
